In [1]:
import numpy as np
import torch 

import pennylane as qml
from pennylane.operation import Operation, AnyWires

#dev0 = qml.device("default.qubit",wires=3)
print(torch.cuda.is_available())
class RBSGate(Operation):
    num_wires = 2  

    def __init__(self, theta, wires, id=None):
        all_wires = qml.wires.Wires(wires)
        super().__init__(theta, wires=all_wires, id=id)

    @staticmethod
    def compute_decomposition(theta, wires):
        decomp = [
                qml.Hadamard(wires=wires[0]),
                qml.Hadamard(wires=wires[1]),
                qml.CZ(wires=wires),
                qml.RY(theta/2.,wires=wires[0]),
                qml.RY(-theta/2.,wires=wires[1]),
                qml.CZ(wires=wires),
                qml.Hadamard(wires=wires[0]),
                qml.Hadamard(wires=wires[1])
            ]
        return decomp
n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def probs_single(inputs, weights):
    shape=[3,1]
    
    qml.PauliX(wires=0)
    # make the input vector norm1
    
    # load input
    prd_fact=1.0
    for qi in range(0,shape[0]-1):
        theta_i=torch.arccos((inputs[...,qi])/prd_fact)#removed np.sqrt
        prd_fact=prd_fact*torch.sin(theta_i)
#        print(f"qubit: {qi}, input: {inputs[...,qi]}, fact_new: {prd_fact}, theta_i: {theta_i}")
#        input()
        RBSGate(theta_i,wires=[qi,qi+1],id=f"$\\alpha_{qi}$")
    
    # parametric circuit
    ctr=0
    for j in range(shape[1]):
        for i in range(0,shape[0]-j-1):
            RBSGate(weights[0][ctr],[i,i+1],id=f"$\\theta_{ctr}$")
            ctr+=1    
    return qml.probs(wires=range(shape[0]-shape[1],shape[0]))
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import make_moons
import wandb

wandb.login()

# Set random seeds
#torch.manual_seed(42)
#np.random.seed(42)

X, y = make_moons(n_samples=200, noise=0.1)

X_mean, X_std=np.mean(X,axis=0), np.std(X,axis=0,ddof=1)

X=(X-X_mean)/X_std

X=np.hstack((X,np.ones(X.shape[0])[:,None]))
X=np.array([np.clip(row/np.sqrt(np.sum(row**2)),-1,1) for row in X])
def stereo_pj(X):
    n,m=np.shape(X)
    newX=np.zeros((n,m+1))
    for rowindex,x in enumerate(X):
        s=np.sum(pow(x,2))
        for index in range(m):
            newX[rowindex,index]=2*x[index]/(s+1)
        newX[rowindex,m]=(s-1)/(s+1)
    return newX

#X=stereo_pj(X)
y_ = torch.unsqueeze(torch.tensor(y,requires_grad=False), 1)  # used for one-hot encoded labels
y_hot = torch.scatter(torch.zeros((200, 2),requires_grad=False), 1, y_, 1)

c = ["#1f77b4" if y_ == 0 else "#ff7f0e" for y_ in y]  # colours for each class
#plt.axis("off")
#plt.scatter(X[:, 0], X[:, 1], c=c)
#plt.show()

n_layers = 1
n_pars = 2
weight_shapes = {"weights": (n_layers, n_pars)}
qlayer = qml.qnn.TorchLayer(probs_single, weight_shapes)
clayer_1 = torch.nn.Linear(3, 3)

softmax_pre = torch.nn.Softmax(dim=1)
softmax = torch.nn.Softmax(dim=1)

optimizers=["SGD","ADAM","ADAMW","RMSPROP"]
epochs_list = [10, 50, 100]
lr_list=[0.1,0.2,0.01,0.001,0.5,0.9]
layers = [qlayer, softmax]

for optindex in optimizers:
    for epochs in epochs_list:
        for lr in lr_list:
                        
            model = torch.nn.Sequential(*layers)
            #print(model)
            
            #print([pm for pm in model.parameters()])
            match optindex:
                case "SGD":
                     opt = torch.optim.SGD(model.parameters(), lr=lr)
                case "ADAM":
                     opt = torch.optim.Adam(model.parameters(), lr=lr)
                case "ADAMW":
                     opt = torch.optim.AdamW(model.parameters(), lr=lr)
                case "RMSPROP":
                     opt = torch.optim.RMSprop(model.parameters(), lr=lr)

            
            loss = torch.nn.L1Loss()
            
            X = torch.tensor(X, requires_grad=False).float()
            y_hot = y_hot.float()
            
            batch_size = 1
            batches = 200 // batch_size
            
            #print(X)
            data_loader = torch.utils.data.DataLoader(
                list(zip(X, y_hot)), batch_size=batch_size, shuffle=False, drop_last=True
            )
            
            
            run = wandb.init(
                # Set the project where this run will be logged
                project="QPNN_Torch",
                entity="quantum_kets",
                # Track hyperparameters and run metadata
                config={
                    "learning_rate": lr,
                    "epochs": epochs,
                    "batch_size": batch_size,
                    "dataset": "moon",
                    "optimizer": optindex,
                    "structure": "q-smax",
                    "achitecture": "extrawire",
                    "normalization": "stdnorm"
                },
            )
            for epoch in range(epochs):
            
                running_loss = 0
            
                for xs, ys in data_loader:
                    opt.zero_grad()
            
            #        print(xs)
                    #print(model(xs))
                    #print(ys)
                    
                    loss_evaluated = loss(model(xs), ys)
                    loss_evaluated.backward()
            
                    opt.step()
            
                    running_loss += loss_evaluated
                
                avg_loss = running_loss / batches
                print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))
                wandb.log({ "loss": avg_loss})
                input()
            
            y_pred = model(X)
            predictions = torch.argmax(y_pred, axis=1).detach().numpy()
            
            correct = [1 if p == p_true else 0 for p, p_true in zip(predictions, y)]
            accuracy = sum(correct) / len(correct)
            print(f"Accuracy: {accuracy * 100}%")
            wandb.log({"accuracy":accuracy * 100})
            

True


wandb: Currently logged in as: giacomoantonioli (quantum_kets). Use `wandb login --relogin` to force relogin


Average loss over epoch 1: 0.4965


Average loss over epoch 2: 0.4689


Average loss over epoch 3: 0.4223


...

accuracy,▁
loss,█▆▃▁▁▁▁▁▁▁
accuracy,85.0
loss,0.40123


Average loss over epoch 1: 0.4019


Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\threading.py", line 982, in run


KeyboardInterrupt: Interrupted by user

    self._target(*self._args, **self._kwargs)
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\site-packages\wandb\sdk\wandb_run.py", line 322, in check_internal_messages
    self._loop_check_status(
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\site-packages\wandb\sdk\wandb_run.py", line 233, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\site-packages\wandb\sdk\interface\interface.py", line 892, in deliver_internal_messages
    return self._deliver_internal_messages(internal_message)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\giaco\anaconda3\envs\qiskit46\Lib\site-packages\wandb\sdk\interface\interface_shared.py", line 510, in _deliver_internal_messages
wandb: While tearing down the service manager. The following error has occurred: [WinError 10054] Connessione in corso interrotta forzatamente dall'host remoto
    return self._deliver_record(record)
          